<a href="https://colab.research.google.com/github/10udCryp7/TV-command-synthesis/blob/main/src_prototype/Phase1_Main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# config of data_preparation

content_list = [
      ('app', '1NIPjwYGi5b1xuB_e8y1ks_oGZ_obalny'),
      ('movie', '19xDdO_BZmpZLHlbXN-aNuEFU_xvqzFjn'),
      ('song', '1-_bVYQUV8DJ4KvG5j7cK0ZcvJY6s9dre'),
      ('tv', '1lZcb1cKaYwCS9wqHf8PXct-cpxAvM2Oz')
]
command_source = '1zHhi08gxGqc_qGTxoQH-YIP7vuzmu7v3'

In [ ]:
# prompt
non_active_prompt = """
You are to generate {generated_nums} sentences of type "non_active".

**Definition of "non_active":**
- Sentences represent natural, everyday indoor human conversation.
- Absolutely no TV/device commands.
- Mentioning or talking about a command is still considered non_active — only direct device instructions count as active.

**Input content:**
- Use the following content list (after removing prefixes): {content_list}

**Rules:**
1. Use only human-to-human conversation, not device commands.
2. Even if the sentence contains TV-related words, if it is not a direct command to the device, it is still non_active.
3. Sentences must sound natural and conversational.
4. Use the specific content ideas from the list.

**Output format (JSON only):**
```json
{{
  "non_active": [
    {{ "text": "...",
  ]
}}
"""
single_active_prompt = """
You are to generate {generated_nums} sentences of type "single_active".

**Definition of "single_active":**
- Each sentence is a direct, clear TV/device command.
- No human conversation or extra wording beyond the command.
- The command must be something that could be spoken to a device to perform an action immediately.

**Input:**
- TV commands list: {command_list}
- Content list (prefix removed): {content_list}

**Rules:**
1. Each sentence = exactly one direct TV/device command from the TV commands list.
2. Do not use questions, hypotheticals, or descriptions — only imperative commands.
3. No unrelated conversation or comments.
4. Must sound natural as a spoken device instruction.

**Output format (JSON only):**
```json
{{
  "single_active": [
    {{ "text": "...",
  ]
}}

"""
single_mix_prompt = """
You are to generate {generated_nums} sentences of type "single_mix".

**Definition of "single_mix":**
- Each sentence must contain exactly ONE direct, clear TV/device command + one unrelated human conversation.
- The command must be an imperative statement addressed to the device, not a question or discussion.
- The two parts must be completely unrelated.

**Position requirement:**
- In this task, the TV/device command should appear {command_position} in the sentence.

**Input:**
- TV commands list: {command_list}
- Content list (prefix removed): {content_list}

**Rules:**
1. Exactly one direct TV/device command + one unrelated human conversation.
2. The command must be in imperative form (telling the device to do something immediately).
3. No hypotheticals, descriptions, or indirect mentions of commands.
4. Commands and conversation must follow the specified position rule above.
5. Sentences must sound natural.
6. When splitting into segments, include every single word from the original sentence — no removals.

**Output format (JSON only):**
```json
{{
  "single_mix": [
    {{
      "text": "...",
      "segments": [
        {{ "0": "...", "type": "active" }},
        {{ "1": "...", "type": "non_active" }}
        // More segments if needed, but all words from the original sentence must be included
      ]
    }}
  ]
}}
"""
chain_active_prompt = """
You are to generate {generated_nums} sentences of type "chain_active".

**Definition of "chain_active":**
- Sentences contain multiple direct TV/device commands only.
- All commands must be imperative instructions to the device.

**Input:**
- TV commands list: {command_list}
- Content list (prefix removed): {content_list}

**Rules:**
1. Each sentence must have two or more imperative device commands from the provided list.
2. No human conversation, questions, or descriptive phrases.
3. Commands can be related or unrelated, but all must be valid direct instructions.
4. Sentences should sound natural as a spoken command sequence.

**Output format (JSON only):**
```json
{{
  "chain_active": [
    {{
      "text": "...",
  }}
  ]
}}
"""
chain_mix_prompt = """
You are to generate {generated_nums} sentences of type "chain_mix".

**Definition of "chain_mix":**
- Sentences must contain multiple direct TV/device commands + at least one unrelated human conversation.
- Commands must be imperative instructions to the device, not questions or descriptions.

**Position requirement:**
- In this task, the sequence of TV/device commands should appear {command_position} in the sentence.

**Input:**
- TV commands list: {command_list}
- Content list (prefix removed): {content_list}

**Rules:**
1. Each sentence must contain 2 or more direct imperative commands from the TV commands list.
2. Must also contain at least one unrelated human conversation segment.
3. Commands and conversation must follow the specified position rule above.
4. No hypotheticals, indirect mentions, or descriptions — only direct instructions are active.
5. Sentences must sound natural.
6. Segments must cover the entire sentence with no missing words or characters.
7. In segments, `"type"` must be `"active"` for commands and `"non_active"` for conversation.

**Output format (JSON only):**
```json
{{
  "chain_mix": [
    {{
      "text": "...",
      "segments": [
        {{ "0": "...", "type": "active" }},
        {{ "1": "...", "type": "non_active" }}
        // More segments as needed, but must cover 100% of the sentence
      ]
    }}
  ]
}}
"""
PROMPT_CORPUS = {
    "non_active": non_active_prompt,
    "single_active": single_active_prompt,
    "single_mix": single_mix_prompt,
    "chain_active": chain_active_prompt,
    "chain_mix": chain_mix_prompt
}

list_position = [
  "at the beginning",
  "at the end",
  "in the middle",
  "scattered throughout",
  "before the conversation part",
  "after the conversation part",
  "surrounding the conversation part",
  "split before and after the conversation part",
  "alternating with conversation parts",
  "mixed randomly with conversation parts"
]

In [ ]:
# data_preparation.py
'''
for loading data

design kiểu này bị cái phải chuẩn format và phải tự chuẩn bị data,
cơ mà scale lên tính sau, miễn fix output là list string là được.

'''
import os
import gdown
import pandas as pd

class DataPreparation:
  def __init__(self, content_list = None, command_source = None, save_dir = 'data'):
    self.content_list = content_list # (Topic, ID on google drive)
    self.command_source = command_source
    self.save_dir = save_dir

    os.makedirs(self.save_dir, exist_ok=True)
    os.makedirs(os.path.join(self.save_dir, 'content'), exist_ok = True)
    os.makedirs(os.path.join(self.save_dir, 'command'), exist_ok = True)


  def download_content(self) -> list[str]:
    list_content_info = []
    try:
      for content_name, content_source in self.content_list:
        url = f'https://drive.google.com/uc?id={content_source}'

        output_path = os.path.join(self.save_dir, 'content', f'{content_name}.csv')
        gdown.download(url, output_path, quiet=False)

        list_content_info.append((content_name, output_path))
      return list_content_info

    except Exception as e:
          print(f"Error downloading {content_source}: {e}")


  def download_command(self) -> str:
    try:
      url = f'https://drive.google.com/uc?id={self.command_source}'

      output_path = os.path.join(self.save_dir, 'command', 'command.csv')
      gdown.download(url, output_path, quiet=False)

      return output_path # expect csv format

    except:
      print(f"Error downloading {self.command_source}")


  def get_content(self, list_content_info) -> dict:
    content_dict = {}
    for type_name, content_path in list_content_info:
        df = pd.read_csv(content_path)
        contents = [f"{row['content']}" for _, row in df.iterrows()]
        content_dict[type_name] = contents

    return content_dict

  def get_command(self, command_path) -> list:
    df = pd.read_csv(command_path)
    commands = [f"{row['command']}" for _, row in df.iterrows()]

    return commands

  def run_pipeline(self, download_content = True, download_command = True):
    if download_content:
      list_content_info = self.download_content()
    else:
      content_dir = os.path.join(self.save_dir, "content")
      list_content_info = [
          (os.path.splitext(f)[0], os.path.join(content_dir, f))
          for f in os.listdir(content_dir)
      ]

    if download_command:
      command_path = self.download_command()
    else:
      command_path = os.path.join(self.save_dir, 'command', 'command.csv')

    content_dict = self.get_content(list_content_info)
    commands = self.get_command(command_path)

    return content_dict, commands

In [ ]:
import random

class PromptGenerator:
  def __init__(self, num_samples_command: int, num_samples_content: int,
               chain_length: int, prompt_dir: str, prompt_types: list,
               list_corpus: dict[list[str]], list_command: list[str],
               list_position: list[str], generated_nums: int, prompt_dict: dict):
    self.num_samples_command = num_samples_command
    self.num_samples_content = num_samples_content
    self.chain_length = chain_length # for chain case
    self.prompt_dir = prompt_dir
    self.prompt_types = prompt_types
    self.list_corpus = list_corpus
    self.list_command = list_command
    self.list_position = list_position
    self.generated_nums = generated_nums
    self.prompt_dict = prompt_dict

  def content_sample(self):
    # corpus = list of list
    sample_list = []
    # sampling content from each data corpus
    for corpus in self.list_corpus:
      sample_list.append(random.sample(self.list_corpus[corpus], self.num_samples_content))

    return sample_list

  def command_sample(self):
    sample = random.sample(self.list_command, self.num_samples_command)
    return sample

  def get_prompt(self, prompt_type, sampled_command, sampled_contents):
    prompt_dict = self.prompt_dict # get from config

    prompt_template = prompt_dict[prompt_type]
    if prompt_type in ("single_mix", "chain_mix"):
      command_position = self.position_sample()
      prompt = prompt_template.format(command_list = sampled_command,
                                      content_list = sampled_contents,
                                      generated_nums = self.generated_nums,
                                      command_position = command_position)
    else:
      prompt = prompt_template.format(command_list = sampled_command,
                                      content_list = sampled_contents,
                                      generated_nums = self.generated_nums)

    return prompt

  def position_sample(self):
    sample = random.sample(self.list_position, 3)

    joiners = ["and/or" for _ in range(3)]
    sample = [f"{pos} {joiner}" for pos, joiner in zip(sample, joiners)]
    sample = " ".join(sample)
    return sample

  def generate_prompt(self):
    prompts = {}
    for prompt_type in self.prompt_types:
      sampled_command = self.command_sample()
      sampled_contents = self.content_sample()

      prompt = self.get_prompt(prompt_type,
                              sampled_command = sampled_command,
                              sampled_contents = sampled_contents)

      prompts[prompt_type] = prompt
    return prompts


In [ ]:
from openai import OpenAI
import os
from google.colab import userdata
import uuid

class TextGenerator:
  def __init__(self, model, temperature):
    self.model = model
    self.temperature = temperature # sau có thể sửa thành 1 file config

    try:
      api_key = self.get_api_key("colab")
      self.client = OpenAI(api_key=api_key)

    except Exception as e:
      print(f'fail at init model!')
      raise e

  def get_api_key(self, env_type: str):
    match env_type:
      case "colab":
        return userdata.get("OPENAI_API_KEY")
      case "local":
        return os.getenv("OPENAI_API_KEY")
      # add more case there

  def generate(self, prompt):
    # just for openai
    response = self.client.responses.create(
        model=self.model,
        input = prompt,
        temperature=self.temperature,
    )
    return response.output[0].content[0].text

  def create_df(self, parsed_data):
    rows = []

    for main_type, items in parsed_data.items():
        for idx, item in enumerate(items):

            id = uuid.uuid4().hex[:8]

            if isinstance(item, dict):
                # Structured format with segments
                text = item.get('text', '')
                segments = item.get('segments', [])

                rows.append({
                    'id': id,
                    'text': text,
                    'type': main_type,
                    'segments': segments if segments else None,
                    'has_segments': len(segments) > 0
                })
            else:
                # Simple string format (fallback)
                rows.append({
                    'id': id,
                    'text': item,
                    'type': main_type,
                    'segments': None,
                    'has_segments': False
                })

    df = pd.DataFrame(rows)

    return df

In [ ]:
# utils.py

import json
import re

def clean_json_string(raw_output):
    # Bỏ phần ```json và ```
    cleaned = re.sub(r"^```json\s*|\s*```$", "", raw_output.strip())
    return cleaned

def parse_generated_json(raw_output):
    cleaned = clean_json_string(raw_output)
    return json.loads(cleaned)

In [ ]:
# main config
prompt_gen_param = {
    "num_samples_command": 5,
    "num_samples_content": 5,
    "chain_length": 3,
    "prompt_dir": "prompt",
    "prompt_types": ['non_active', 'single_active', 'single_mix', 'chain_active', 'chain_mix'],
    "generated_nums": 4,
}

text_gen_param = {
    "model": "gpt-4o-mini",
    "temperature": 1,
}

LOOP = 5
FILE_NUM = 50
START = 0

In [ ]:
# main.py
import itertools
from tqdm import tqdm
import os

def main(download = True, export_folder = 'command_text'):
  os.makedirs(export_folder, exist_ok = True)
  dp = DataPreparation(content_list = content_list,
                       command_source = command_source)

  if download:
    content_dict, commands = dp.run_pipeline()
  else:
    content_dict, commands = dp.run_pipeline(download_content = False, download_command = False)

  pg = PromptGenerator(list_corpus = content_dict,
                       list_command = commands,
                       list_position = list_position,
                       prompt_dict= PROMPT_CORPUS,
                       **prompt_gen_param)

  tg = TextGenerator(**text_gen_param)

  list_command = []

  for i in tqdm(range(START, START + FILE_NUM)):
      for _ in range(LOOP):
          prompts = pg.generate_prompt()
          for prompt_type, prompt in prompts.items():
              generated_output = tg.generate(prompt)
              parsed_data = parse_generated_json(generated_output)

              df_commands = tg.create_df(parsed_data)
              list_command.append(df_commands)

      list_command_df = pd.concat(list_command, ignore_index=True)
      path = os.path.join(export_folder, f'commands_{i}.csv')
      list_command.to_csv(path, idnex=False)
    list_command_df.to_csv('synthesis_command.csv', index=False)


In [ ]:
main(download = True)

Downloading...
From: https://drive.google.com/uc?id=1NIPjwYGi5b1xuB_e8y1ks_oGZ_obalny
To: /content/data/content/app.csv
100%|██████████| 622/622 [00:00<00:00, 1.76MB/s]
Downloading...
From: https://drive.google.com/uc?id=19xDdO_BZmpZLHlbXN-aNuEFU_xvqzFjn
To: /content/data/content/movie.csv
100%|██████████| 250k/250k [00:00<00:00, 9.50MB/s]
Downloading...
From: https://drive.google.com/uc?id=1-_bVYQUV8DJ4KvG5j7cK0ZcvJY6s9dre
To: /content/data/content/song.csv
100%|██████████| 2.50M/2.50M [00:00<00:00, 38.1MB/s]
Downloading...
From: https://drive.google.com/uc?id=1lZcb1cKaYwCS9wqHf8PXct-cpxAvM2Oz
To: /content/data/content/tv.csv
100%|██████████| 6.27k/6.27k [00:00<00:00, 11.3MB/s]
Downloading...
From: https://drive.google.com/uc?id=1zHhi08gxGqc_qGTxoQH-YIP7vuzmu7v3
To: /content/data/command/command.csv
100%|██████████| 1.35k/1.35k [00:00<00:00, 3.10MB/s]
100%|██████████| 50/50 [1:30:44<00:00, 108.89s/it]
